# Process railways from OSM

In [ ]:
import os
from glob import glob

import fiona
import geopandas as gpd

In [ ]:
files = glob('../incoming_data/OpenStreetMap/railways/*rail.gpkg')

In [ ]:
def process_points(df):
    print("Railway", df.railway.unique())

    df = df[ 
        df['railway'].isin(('stop', 'station', 'halt', 'yes')) 
    ]

    return df[['osm_id', 'name', 'railway', 'geometry']]

In [ ]:
def process_lines(df):
    print("Railway", df.railway.unique())


    df = df[ 
        ~df['railway'].isin(('abandoned', 'disused', 'construction', 'proposed', 'tram', 'funicular')) 
    ]

    return df[['osm_id', 'name', 'railway', 'bridge', 'geometry']]

In [ ]:
def process_polygons(df):
    print("Railway", df.railway.unique())
    print("Disused", df.disused.unique())
    

    df = df[
        ~df['railway'].isin(('construction', 'proposed'))
        & ~df['disused'].isin(('yes',))
    ]

    return df[['osm_id', 'osm_way_id', 'name', 'railway', 'geometry']]

In [ ]:
def polys_to_points(df):
    df.geometry = df.geometry.centroid
    return df

In [ ]:
for fname in files:
    print(f"\n{fname}")
    layers = fiona.listlayers(fname)
    out_fname = fname.replace('.gpkg', '_filtered.gpkg')
    try:
        os.remove(out_fname)
    except FileNotFoundError:
        pass
    
    if 'points' in layers:
        df = process_points(gpd.read_file(fname, layer='points'))
        if not df.empty:
            df.to_file(out_fname, layer='points', driver="GPKG")
    
    if 'lines' in layers:
        df = process_lines(gpd.read_file(fname, layer='lines'))
        if not df.empty:
            df.to_file(out_fname, layer='lines', driver="GPKG")
        
    if 'multipolygons' in layers:
        df = process_polygons(gpd.read_file(fname, layer='multipolygons'))
        if not df.empty:
            df.to_file(out_fname, layer='multipolygons', driver="GPKG")
            
            df = polys_to_points(df)
            df.to_file(out_fname, layer='centroids', driver="GPKG")